In [3]:
#task 1

PIECE_VALUES = {'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 0,  # White pieces
                'p': -1, 'n': -3, 'b': -3, 'r': -5, 'q': -9, 'k': 0}  # Black pieces

INITIAL_BOARD = {
    "e2": "P", "d7": "p", "g1": "N", "b8": "n",  # Pawns and knights
    "e1": "K", "e8": "k", "d1": "Q", "d8": "q",  # Kings and Queens
}

def evaluate_board(board):
    """Basic evaluation: sum of piece values."""
    return sum(PIECE_VALUES.get(piece, 0) for piece in board.values())

def generate_moves(board, is_white):
    """
    Generates simple pseudo-moves for pawns and knights.
    Only considers a few valid moves for simplicity.
    """
    moves = []
    for pos, piece in board.items():
        if is_white and piece == "P":  # White pawn
            new_pos = pos[0] + str(int(pos[1]) + 1)
            if new_pos not in board:
                moves.append((pos, new_pos))
        elif not is_white and piece == "p":  # Black pawn
            new_pos = pos[0] + str(int(pos[1]) - 1)
            if new_pos not in board:
                moves.append((pos, new_pos))
        elif piece in "Nn":  # Knights
            knight_moves = [(2, 1), (1, 2), (-1, 2), (-2, 1),
                            (-2, -1), (-1, -2), (1, -2), (2, -1)]
            for dx, dy in knight_moves:
                new_file = chr(ord(pos[0]) + dx)
                new_rank = str(int(pos[1]) + dy)
                new_pos = new_file + new_rank
                if "a" <= new_file <= "h" and "1" <= new_rank <= "8" and (new_pos not in board or board[new_pos].islower() != is_white):
                    moves.append((pos, new_pos))

    return moves

def apply_move(board, move):
    new_board = board.copy()
    start, end = move
    new_board[end] = new_board.pop(start)  # Move piece
    return new_board

def beam_search(board, beam_width, depth_limit, is_white=True):
    beam = [(board, [], evaluate_board(board))]  # (board, move_sequence, score)

    for _ in range(depth_limit):
        candidates = []

        for state, moves, score in beam:
            legal_moves = generate_moves(state, is_white)
            for move in legal_moves:
                new_state = apply_move(state, move)
                new_score = evaluate_board(new_state)
                candidates.append((new_state, moves + [move], new_score))

        beam = sorted(candidates, key=lambda x: x[2], reverse=is_white)[:beam_width]

    return beam[0][1], beam[0][2]

best_moves, eval_score = beam_search(INITIAL_BOARD, beam_width=2, depth_limit=2)
print("Best Move Sequence -> ", [f"{start}->{end}" for start, end in best_moves])
print("Evaluation Score -> ", eval_score)


Best Move Sequence ->  ['e2->e3', 'g1->h3']
Evaluation Score ->  0


In [4]:
#task 2

import random
import math

def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def total_distance(route):
    return sum(distance(route[i], route[i + 1]) for i in range(len(route) - 1)) + distance(route[-1], route[0])

def hill_climb(locations, max_iterations=1000):
    current_route = locations[:]
    random.shuffle(current_route)
    current_distance = total_distance(current_route)

    for _ in range(max_iterations):
        i, j = sorted(random.sample(range(len(locations)), 2))
        new_route = current_route[:]
        new_route[i:j + 1] = reversed(new_route[i:j + 1])
        new_distance = total_distance(new_route)

        if new_distance < current_distance:
            current_route, current_distance = new_route, new_distance

    return current_route, current_distance

locations = [(0, 0), (2, 3), (5, 4), (7, 8), (6, 1), (3, 6)]
optimized_route, total_dist = hill_climb(locations)

print("Optimized Route ->", optimized_route)
print("Total Distance -> ", total_dist)


Optimized Route -> [(0, 0), (6, 1), (5, 4), (7, 8), (3, 6), (2, 3)]
Total Distance ->  24.957141036098122


In [5]:
#task 3

import random
import math

def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def total_distance(route):
    return sum(distance(route[i], route[i + 1]) for i in range(len(route) - 1)) + distance(route[-1], route[0])

def create_population(cities, size):
    return [random.sample(cities, len(cities)) for _ in range(size)]

def select_parents(population):
    return sorted(population, key=total_distance)[:2]

def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = parent1[start:end + 1] + [city for city in parent2 if city not in parent1[start:end + 1]]
    return child

def mutate(route, mutation_rate=0.1):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]
    return route

def genetic_algorithm(cities, population_size=100, generations=500):
    population = create_population(cities, population_size)

    for _ in range(generations):
        parents = select_parents(population)
        offspring = [mutate(crossover(parents[0], parents[1])) for _ in range(population_size)]
        population = parents + offspring

    best_route = min(population, key=total_distance)
    return best_route, total_distance(best_route)

cities = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(10)]
best_route, best_distance = genetic_algorithm(cities)

print("Best Route -> ", best_route)
print("Best Distance -> ", best_distance)


Best Route ->  [(14, 57), (17, 24), (10, 16), (95, 12), (95, 22), (75, 48), (98, 77), (22, 88), (3, 99), (12, 75)]
Best Distance ->  351.1654510139921
